<a href="https://colab.research.google.com/github/RileyWClarke/MLTSA_RClarke/blob/master/HW9/mltsa_hw9_rclarke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from matplotlib import pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
import pandas as pd
% matplotlib inline

In [3]:
from keras.models import Sequential
from keras.layers import Dense#, Dropout, Flatten
#from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
plt.style.use("https://raw.githubusercontent.com/fedhere/MLTSA_FBianco/master/fbb.mplstyle")

In [4]:
!pip install kaggle

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
cd gdrive/My\ Drive/

/content/gdrive/My Drive


In [7]:
ls kaggle.json

kaggle.json


In [0]:
!chmod 600 kaggle.json

In [0]:
import json
import os
envs = json.load(open("kaggle.json", "r"))
os.environ["KAGGLE_USERNAME"] = envs['username']
os.environ["KAGGLE_KEY"] = envs['key']

In [10]:
!kaggle competitions list -s PLAsTiCC

ref            deadline             category   reward  teamCount  userHasEntered  
-------------  -------------------  --------  -------  ---------  --------------  
PLAsTiCC-2018  2018-12-17 23:59:00  Featured  $25,000       1094            True  


In [11]:
cd plasticc

/content/gdrive/My Drive/plasticc


In [12]:
cd data

/content/gdrive/My Drive/plasticc/data


In [0]:
#!kaggle competitions download PLAsTiCC-2018

test_set_batch7.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch11.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
training_set.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch5.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
data_note.pdf.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch3.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
training_set_metadata.csv: Skipping, found more recently modified local copy (use --force to force download)
test_set.csv.zip: Skipping, found more re

In [0]:
ls

allfeatures                test_set_batch2.csv.zip  test_set.csv.zip
allfeatures.npy            test_set_batch3.csv.zip  test_set_metadata.csv.zip
data_note.pdf.zip          test_set_batch4.csv.zip  test_set_sample.csv.zip
metadata                   test_set_batch5.csv.zip  training_set.csv
sample_submission.csv.zip  test_set_batch6.csv.zip  training_set.csv.zip
test_set_batch10.csv.zip   test_set_batch7.csv.zip  training_set_metadata.csv
test_set_batch11.csv.zip   test_set_batch8.csv.zip
test_set_batch1.csv.zip    test_set_batch9.csv.zip


In [0]:
!unzip training_set.csv.zip

Archive:  training_set.csv.zip
replace training_set.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: training_set.csv        


In [0]:
lcvs = pd.read_csv('training_set.csv')
lcvs.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [0]:
starts = lcvs.groupby(["object_id"])[["mjd"]].min()
starts

,mjd
object_id,
615,59750.4229
713,59825.2600
730,59798.3205
745,59770.3662
1124,59750.4229
...,...
130739978,59790.3778
130755807,59581.3605
130762946,59620.2130


In [0]:
starts['mjd'].loc[starts.index == 615].values[0]

59750.4229

In [0]:
lcvs['object_id'].values[0]

615

Create new domain where each object is time indexed with its first observation being zero

In [0]:
#Very stupid and expensive way to do this:

mjd_new = np.zeros(lcvs.shape[0])

for i, obj in enumerate(lcvs['object_id'].values):

  if lcvs['mjd'][i] == starts['mjd'].loc[starts.index == obj].values[0]:
    pass

  else:
    mjd_new[i] = lcvs['mjd'][i] - starts['mjd'].loc[starts.index == obj]

 
mjd_new

array([0.000000e+00, 7.700000e-03, 1.540000e-02, ..., 8.606095e+02,
       8.755844e+02, 8.776242e+02])

In [0]:
lcvs['mjd_new'] = mjd_new
lcvs.head()

,object_id,mjd,passband,flux,flux_err,detected,mjd_new
0,615,59750.4229,2,-544.810303,3.622952,1,0.0000
1,615,59750.4306,1,-816.434326,5.553370,1,0.0077
2,615,59750.4383,3,-471.385529,3.801213,1,0.0154
3,615,59750.4450,4,-388.984985,11.395031,1,0.0221
4,615,59752.4070,2,-681.858887,4.041204,1,1.9841


In [0]:
lcvs.to_pickle('lcvs.pkl')

In [0]:
lcvs = pd.read_pickle('lcvs.pkl')

In [26]:
lcvs_p1 = lcvs.loc[lcvs['passband'].values == 1]
lcvs_p1.head()

,object_id,mjd,passband,flux,flux_err,detected,mjd_new
1,615,59750.4306,1,-816.434326,5.553370,1,0.0077
5,615,59752.4147,1,-1061.457031,6.472994,1,1.9918
10,615,59767.3045,1,-815.188599,5.293019,1,16.8816
15,615,59770.2256,1,-820.042786,5.875329,1,19.8027
20,615,59779.3265,1,-921.002502,6.306800,1,28.9036


Flatten time series

In [0]:
lcvsp1_flat = np.concatenate((lcvs_p1['mjd_new'].values, lcvs_p1['flux'].values))#This vector gets fed to autoencoder

In [33]:
lcvsp1_flat

array([ 7.7000000e-03,  1.9918000e+00,  1.6881600e+01, ...,
       -1.3734149e+01,  2.9869080e+00,  1.4894439e+01])

In [34]:
N = len(lcvsp1_flat)
N

352998

In [0]:
from sklearn.model_selection import train_test_split
X_tr,  X_te, y_tr, y_te = train_test_split(lcvsp1_flat, lcvsp1_flat, 
                                             test_size = .25, random_state=1)

In [55]:
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((264748,), (88250,), (264748,), (88250,))

In [0]:
def create_model(bottleneck=128, input_dim=1225):
  model = Sequential()
  model.add(Dense(256, activation='relu', input_dim=input_dim))
  model.add(Dense(128, activation='relu', name="bottleneck"))
  if bottleneck < 128:
    model.add(Dense(bottleneck, activation='relu'))
    model.add(Dense(128, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(input_dim, activation='relu'))
  model.compile(optimizer='adamax', loss='mse',
              metrics=['mae', 'acc'])# optimizer="adadelta", loss="kullback_leibler_divergence")
  return model
model = create_model(input_dim=N)

In [21]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               90367744  
_________________________________________________________________
bottleneck (Dense)           (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 352998)            90720486  
Total params: 181,154,150
Trainable params: 181,154,150
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit?

In [67]:
!wget http://fbb.space/mltsa/imgreconstruct_10000.h5
model.load_weights("imgreconstruct_10000.h5")

--2020-05-08 01:44:50--  http://fbb.space/mltsa/imgreconstruct_10000.h5
Resolving fbb.space (fbb.space)... 107.180.39.232
Connecting to fbb.space (fbb.space)|107.180.39.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80457300 (77M)
Saving to: ‘imgreconstruct_10000.h5’

imgreconstruct_1000 100%[===================>]  76.73M  23.8MB/s    in 3.6s    

2020-05-08 01:44:54 (21.2 MB/s) - ‘imgreconstruct_10000.h5’ saved [80457300/80457300]



ValueError: ignored